---------------------------------------------------------

# PhAst Algorithm (Part 4 of 5): 

#Generating Phase Curves to Determine the Asteroids Absolute Magnitude and Slope Parameter 

## By Arushi Nath

----------------------------------------------

#### This Fourth Notebook will walk you through the steps of Determining the Reduced Magnitude of the asteroid. This will allow for the creation of phase curves to yield asteroid physical properties.


----------------------------------------------

# Sparse Photometry on Asteroids

### 1. Importing Required Libraries

In [ ]:
### Import Necessary Libraries
# Here we import all necessary libraries required for data manipulation, numerical calculations, and plotting. Key libraries include:
# - matplotlib.pyplot for plotting graphs.
# - numpy for numerical operations.
# - astropy for astronomy-specific functions such as handling times and units.
# - pandas for data manipulation and reading CSV files.
# - pyedra and sbpy.photometry for specific asteroid curve modeling.

import matplotlib.pyplot as plt
import numpy as np
from tqdm.notebook import tqdm
from astropy.time import Time
import csv
import matplotlib.pyplot as plt
import juliandate as jd
import requests
import pyedra
from astropy.table import Table
from astropy import units as un
import sbpy.photometry as pm
import scipy.optimize as sco
import glob
import pandas as pd
from csv import writer
import io
import requests


## QUERYING GAIA SPACE BASED SKY SURVEY FOR SERENDIPITOUS ASTEROID OBSERVATIONS

### 2. Querying GAIA to Find Asteroids Magnitude and Associated JD

In [ ]:
### Data Import and Filtering
# Loading observational data from a CSV file and filtering based on the asteroid of interest. 
# Storing relevant data such as solution ID, source ID, and magnitudes in separate lists for further analysis.
solution_id = []
source_id = []
observation_id = []
MPC = []
epoch_utc = []
g_mag = []
g_flux = []
g_flux_error = []

asteroid = '4786'  # Define the asteroid number

with open(r"combined.csv", newline='') as f:
    reader = csv.reader(f)
    for row in reader:
        if row[5] == asteroid:
            if row[17] != 'null':
                solution_id.append(float(row[0]))
                source_id.append(float(row[1]))
                observation_id.append(float(row[4]))
                MPC.append(float(row[5]))
                epoch_utc.append(float(row[8]))
                g_mag.append(float(row[17]))
                g_flux.append(float(row[18]))
                g_flux_error.append(float(row[19]))

Num = len(MPC)  # Count the number of observations for the specified asteroid


In [ ]:
### Conversion to Visual Magnitude
# Convert g-band magnitudes to V-band magnitudes. This cell assumes a direct conversion, which may need adjustment based on specific astronomical standards.



### 3. Taking the Daily Average Magnitude

In [ ]:
### Light Curve Averaging
# Processes the light curve data to average the observations by day. This reduces noise and accounts for any discrepancies in observation times. The tqdm library is used to show progress during processing.
v_mag = []
for i in g_mag:
    v_mag.append(i)  # Directly appending g-band magnitudes to V-band list

n = 0
Mag_av = []
epoch_utc_av = []

Mag_avg = []
epoch_utc_avg = []

e2 = epoch_utc.copy()

while n < len(epoch_utc):
    try:
        a = str(np.min(e2))[0:4]

        for s in tqdm(range(len(epoch_utc))):
            print(str(epoch_utc[s])[0:4], a)
            if str(epoch_utc[s])[0:4] == a:
                Mag_av.append(v_mag[s])
                epoch_utc_av.append(epoch_utc[s])
                n += 1
                e2.remove(epoch_utc[s])
        Mag_avg.append(np.average(Mag_av))
        epoch_utc_avg.append(np.average(epoch_utc_av))


In [ ]:
plt.scatter(epoch_utc_avg, Mag_avg)

In [ ]:
### Convert and Adjust Data for Analysis by Converting to Julian Date

mag = []  # List to store adjusted magnitudes
JD = []   # List to store Julian Dates converted from epoch data

for i in Mag_avg:
    mag.append(i)

# Converting epoch data to Julian Date format by adding a constant offset. This is a common practice in astronomy to align dates with a standard epoch.
# 2455197.5 represents the Julian Date for a specific reference time, which needs to be adjusted based on the epoch data's starting point.
for i in epoch_utc_avg:
    JD.append(i + 2455197.5)

### 4. Applying Unity Offsets to Account for Changing Distances

In [ ]:
### Generate API Link for Asteroid Phase Angle Data
# This function constructs a URL for querying the JPL Horizons system to retrieve phase angle data for a specified asteroid over a given time range.
# The function takes three parameters: the asteroid's identifier, the start time, and the end time for the query.

def PA_finder(asteroid, START_TIME, END_TIME):
    sep = ''  # Separator for joining strings; empty string implies direct concatenation without additional characters.

    # Concatenating parts of the API request with the necessary parameters:
    ast_mod = sep.join("&COMMAND=" + asteroid)  # Appending the asteroid identifier to the command part of the URL.
    start_mod = sep.join("&START_TIME=" + START_TIME)  # Appending the start time for the data query.
    end_mod = sep.join("&STOP_TIME=" + END_TIME)  # Appending the end time for the data query.

    # Base URL for the JPL Horizons system, formatted to request data in text format, querying data specifically for the 'Gaia' barycenter.
    start = "https://ssd.jpl.nasa.gov/api/horizons.api?format=text&CENTER='@gaia'&OBJ_DATA=NO&STEP_SIZE='1min'&QUANTITIES='19,20,24'"

    # Combining all parts to form the complete API link:
    link = start, ast_mod, start_mod, end_mod
    link = sep.join(link)  # Joining all parts into a single URL string.
  
    return link  # Returning the full URL for the API request.


In [ ]:
### Retrieve and Parse Asteroid Data from API
# This code block queries an API for astronomical data of an asteroid at specified Julian Dates (JD),
# then parses and extracts relevant data such as the distance from the Sun, Earth, and the phase angle.

Sun_dist = []    # List to hold the distance from the Sun
Time_ast = []    # List to hold timestamps of the data
Earth_dist = []  # List to hold the distance from Earth
Phase_Angle = [] # List to hold the phase angle

# Loop through each Julian Date to make an API call and process the returned data
for s in tqdm(range(len(JD))):
    # Setting up start and end times for the API request
    START_TIME = str("JD" + str(JD[s]))
    END_TIME = str("JD" + str(JD[s]+ 0.0007))

    # Print the Julian Date to monitor progress
    print(JD[s])

    # Get the API query URL by calling the PA_finder function
    ast_query = PA_finder(asteroid, START_TIME, END_TIME)

    # Making the API request and getting the response
    response_API = requests.get(ast_query)
    data = response_API.text
    
    # Parsing the data received from the API
    a = data.split("*")
    q = []
    for i in a:
        if i != '':
            q.append(i)
    
    # Extracting the required information from the parsed data
    qs = []
    z = q[-3].split(" ")
    for i in z:
        if i != '':
            qs.append(i)
    
    # Appending the extracted data to their respective lists
    Time_ast.append(qs[2] + " " + qs[1])  # Combining date and time for better readability
    Sun_dist.append(float(qs[3]))        # Distance from the Sun
    Earth_dist.append(float(qs[5]))      # Distance from Earth
    Phase_Angle.append(float(qs[7][:-1]))  # Phase angle, removing the last character assumed to be non-numeric


In [ ]:
from astropy.time import Time
t = Time(JD[0], format='jd')
utc = t.to_datetime()

In [ ]:
### Applying Unity Offsets to get the Reduced Magnitude of the Asteroid
Unity = []
for i in range(len(mag)):
    
    a = mag[i] - 5*np.log10(Sun_dist[i]*Earth_dist[i])
    Unity.append(a)

In [ ]:
### Plotting Phase Curve
plt.scatter(Phase_Angle, Unity)
#plt.ylim(8.1,7)
plt.show()

In [ ]:
paired_sorted = sorted(zip(Phase_Angle, Unity))
Phase_Angle_Final, Unity_Final = zip(*paired_sorted)

In [ ]:
plt.scatter(Phase_Angle_Final, Unity_Final)
#plt.ylim(8.1,7)
plt.show()

### 5. Applying Filter Offsets

In [ ]:
### Applying Filter Offsets

t1_GAIA = []
t2_GAIA = []
for i in range(len(Unity_Final)):
    if Phase_Angle_Final[i] <=20:
        a = Unity_Final[i] + 0.02269 - 0.01784*(0.45) + 1.016*(0.45)**2 - 0.2225*(0.45)**3 #V-R
        #a = Unity_Final[i] +0.02907 + 0.02385*(0.684) + 0.2297*(0.684)**2 + 0.001768*(0.684)**3 #B-V
        t1_GAIA.append(Phase_Angle_Final[i])
        t2_GAIA.append(a)
nysa = Table([t1_GAIA,t2_GAIA], names=('phase','V'))
nysa['phase'].unit = 'deg'
nysa['V'].unit = 'mag'
print(nysa)

### 6. Creating H and G Curve

In [ ]:
 # Photometric system from sbpy package, photometry submodule
def pf(xdeg,par1,par2):
    return pm.HG.evaluate(xdeg*np.pi/180,par1,par2)
# Sum-of-squares
def sse_fun(x,data):
    return sum([(pf(d[0],x[0],x[1])-d[1])**2 for d in data])
# Fit function
def fit_fun(data, x0=[6,0.12]):
    cv = ({'type': 'ineq', 'fun': lambda x: x[1]},
    {'type': 'ineq', 'fun': lambda x: 1-x[1]})
    return sco.minimize(sse_fun,x0,args=data,constraints=cv,method='COBYLA')


In [ ]:
res = fit_fun(nysa)
print("RMS of H,G-fit is {:2.8f}".format(res.fun))
print("Fitted H={:1.4f} and G={:1.6f}".format(res.x[0],res.x[1]))

In [ ]:
fx=np.linspace(nysa['phase'][0],nysa['phase'][-1],100)
fy=np.array([pf(x,res.x[0],res.x[1]) for x in fx])
g,p1=plt.subplots(1,1)
p1.plot(nysa['phase'],nysa['V'],'bo')
p1.plot(fx,fy,'r')
p1.set_title('Themis (GAIA)')
p1.set_xlabel('phase angle (deg)')
p1.set_ylabel('G mag')
p1.invert_yaxis()

## QUERYING ZTF ALL SKY SURVEY FOR SERENDIPITOUS ASTEROID OBSERVATIONS

### 1. Querying FINK Server

In [ ]:
r = requests.post(
  'https://fink-portal.org/api/v1/sso',
  json={
    'n_or_d': asteroid,
    'output-format': 'json',
    #'columns': 'i:jd,i:magpsf'
  }
)

# Format output in a DataFrame
pdf = pd.read_json(io.BytesIO(r.content))

In [ ]:
#{"doc":"Filter ID (1=g; 2=R; 3=i)
#V-g = -0.4508 and V-r=0.3379 (with ZTF filters)
JD = []
mag = []
for i in range(len(pdf['i:jd'])):
    if pdf['i:fid'][i]==2:
        JD.append(pdf['i:jd'][i])
    
for i in range(len(pdf['i:magpsf'])):
    if pdf['i:fid'][i]==2:
        
        mag.append((pdf['i:magpsf'][i]))#-0.4508

### 2. Applying Unity Offsets to Get Reduced Magnitude

In [ ]:
def PA_finder(asteroid, START_TIME, END_TIME): 

    sep = ''
    ast_mod = sep.join("&COMMAND=" + asteroid)
    start_mod = sep.join("&START_TIME=" + START_TIME)
    end_mod = sep.join("&STOP_TIME=" + END_TIME)
   
    start = "https://ssd.jpl.nasa.gov/api/horizons.api?format=text&OBJ_DATA=NO&STEP_SIZE='1min'&QUANTITIES='19,20,24'"
    link = start, ast_mod, start_mod, end_mod
    link = sep.join(link)
 
    return link

In [ ]:
Sun_dist = []
Time_ast = []
Earth_dist = []
Phase_Angle = []
for s in tqdm(range(len(JD))):
  
    START_TIME = str("JD" + str(JD[s]))
    END_TIME = str("JD" + str(JD[s]+ 0.0007))
    print(JD[s])
    ast_query = PA_finder(asteroid, START_TIME, END_TIME)
    
    response_API = requests.get(ast_query)
    data = response_API.text
    
    
    a = data.split("*")
    q = []
    for i in a:
        if i != '':
            q.append(i)
        
    qs = []
    z = q[-3].split(" ")
    for i in z:
        if i != '':
            qs.append(i)
    Time_ast.append(qs[2] + " " + qs[1])
    Sun_dist.append(float(qs[3]))
    Earth_dist.append(float(qs[5]))
    Phase_Angle.append(float(qs[7][:-1]))

In [ ]:
from astropy.time import Time
t = Time(JD[0], format='jd')
utc = t.to_datetime()

In [ ]:
plt.scatter(JD, Phase_Angle)

In [ ]:
plt.scatter(Phase_Angle, mag)

In [ ]:
Unity = []
for i in range(len(mag)):
    
    a = mag[i] - 5*np.log10(Sun_dist[i]*Earth_dist[i])
    Unity.append(a)

In [ ]:
plt.scatter(JD, mag)

In [ ]:
plt.scatter(JD, Unity)

In [ ]:
plt.scatter(Phase_Angle, Unity)
#plt.ylim(8.1,7)
plt.show()

In [ ]:
paired_sorted = sorted(zip(Phase_Angle, Unity))
Phase_Angle_Final, Unity_Final = zip(*paired_sorted)

In [ ]:
plt.scatter(Phase_Angle, Unity)
#plt.ylim(8.1,7)
plt.show()

In [ ]:
plt.scatter(Phase_Angle_Final, Unity_Final)
#plt.ylim(8.1,7)
plt.show()

### 3. Applying Filter Offsets

In [ ]:

t2_ZTF = []
t1_ZTF = []
for i in range(len(Unity_Final)):
    if float(Phase_Angle_Final[i]) <=20:
        t2_ZTF.append(Unity_Final[i]+0.3379)
       
        t1_ZTF.append(Phase_Angle_Final[i])
    
nysa = Table([t1_ZTF,t2_ZTF], names=('phase','V'))
nysa['phase'].unit = 'deg'
nysa['V'].unit = 'mag'
print(nysa)

### 4. Creating H and G Curve

In [ ]:
res = fit_fun(nysa)
print("RMS of H,G-fit is {:2.8f}".format(res.fun))
print("Fitted H={:1.4f} and G={:1.6f}".format(res.x[0],res.x[1]))

In [ ]:
fx=np.linspace(nysa['phase'][0],nysa['phase'][-1],100)
fy=np.array([pf(x,res.x[0],res.x[1]) for x in fx])
#g,p1=plt.subplots(1,1)
plt.figure(figsize=(7, 5))
plt.plot(nysa['phase'],nysa['V'],'bo')
plt.plot(fx,fy,'r')
plt.title('Themis (ZTF Telescope)')
plt.xlabel('Phase Angle (deg)')
plt.ylabel('V magnitude')
plt.ylim(max(nysa['V']+0.05), min(nysa['V']-0.1))
plt.show()

## QUERYING ALCDEF OBSERVATIONS

### 1. Querying File for All ALCDEF Observations of Didymos

In [ ]:
g_mag = []
JD2 = []
epoch_utc = []
Error = []
Color = []
with open(r"Tatianina SPARSE.csv", newline='') as f:
  reader = csv.reader(f)
  for row in reader:
      
    print(row)
    
    g_mag.append((row[1]))
    
    epoch_utc.append((row[0]))
    
    
    #Color.append(row[3])
g_mag = g_mag[1:]
g_mag = [eval(i) for i in g_mag]

epoch_utc = epoch_utc[1:]

epoch_utc = [eval(i) for i in epoch_utc]

Error = Error[1:]
Error = [eval(i) for i in Error]

In [ ]:
n = 0
Mag_av = []
epoch_utc_av = []

Mag_avg = []
epoch_utc_avg = []

e2 = epoch_utc.copy()

while n< len(epoch_utc):
    try:
        a = str(np.min(e2))[0:7]
    
        for s in tqdm(range(len(epoch_utc))):
            print(str(epoch_utc[s])[0:7], a)
            if str(epoch_utc[s])[0:7] == a:
    
            
                Mag_av.append(g_mag[s])
                epoch_utc_av.append(epoch_utc[s])
                n+=1
                e2.remove(epoch_utc[s])
        Mag_avg.append(np.average(Mag_av))
        epoch_utc_avg.append(np.average(epoch_utc_av))
        Mag_av = []
        epoch_utc_av = []

    except:
        pass

In [ ]:
plt.scatter(epoch_utc_avg, Mag_avg)

In [ ]:
mag = []
JD = []
for i in Mag_avg:
    mag.append(i)
for i in epoch_utc_avg:
    JD.append(i)

### 2. Applying Unity Offsets to get Reduced Magnitude

In [ ]:
def PA_finder(asteroid, START_TIME, END_TIME): 

    sep = ''
    ast_mod = sep.join("&COMMAND=" + asteroid)
    start_mod = sep.join("&START_TIME=" + START_TIME)
    end_mod = sep.join("&STOP_TIME=" + END_TIME)
   
    start = "https://ssd.jpl.nasa.gov/api/horizons.api?format=text&OBJ_DATA=NO&STEP_SIZE='1min'&QUANTITIES='19,20,24'"
    link = start, ast_mod, start_mod, end_mod
    link = sep.join(link)
 
    return link

In [ ]:
### Collect and Process Astronomical Data
# This code retrieves specific astronomical data for an asteroid at given times and processes it to extract distances and phase angles.

Sun_dist = []    # List to store distances from the Sun
Time_ast = []    # List to store time of each data point
Earth_dist = []  # List to store distances from Earth
Phase_Angle = [] # List to store phase angles

# Loop through each Julian Date to query astronomical data
for s in tqdm(range(len(JD))):
    # Construct API request times based on the Julian Date
    START_TIME = "JD" + str(JD[s])
    END_TIME = "JD" + str(JD[s] + 0.0007)  # Small increment to define a range around the given JD
    print(JD[s])  # Output current Julian Date being processed

    # Generate the API request URL
    ast_query = PA_finder(asteroid, START_TIME, END_TIME)
    
    # Send the API request and capture the response
    response_API = requests.get(ast_query)
    data = response_API.text  # API response text
    
    # Split the response text to parse the relevant data
    a = data.split("*")
    q = [i for i in a if i]  # Filter out empty strings from the split results
    
    # Parse the specific data segment from the response
    qs = []
    z = q[-3].split(" ")  # Select the third-last item from 'q' and split it by spaces
    qs = [i for i in z if i]  # Filter out empty strings from 'z'
    
    # Extract and store the specific data elements
    Time_ast.append(qs[2] + " " + qs[1])  # Combine date and time for better readability
    Sun_dist.append(float(qs[3]))  # Convert the Sun distance to float and store
    Earth_dist.append(float(qs[5]))  # Convert the Earth distance to float and store
    Phase_Angle.append(float(qs[7][:-1]))  # Extract the phase angle, remove the last character, convert to float and store


In [ ]:
from astropy.time import Time
t = Time(JD[0], format='jd')
utc = t.to_datetime()

In [ ]:
plt.scatter(JD, Phase_Angle)

In [ ]:
plt.scatter(Phase_Angle, mag)

In [ ]:
Unity = []
for i in range(len(mag)):
    
    a = mag[i] - 5*np.log10(Sun_dist[i]*Earth_dist[i])
    Unity.append(a)

In [ ]:
plt.scatter(JD, mag)

In [ ]:
plt.scatter(JD, Unity)

In [ ]:
paired_sorted = sorted(zip(Phase_Angle, Unity))
Phase_Angle_Final, Unity_Final = zip(*paired_sorted)

In [ ]:
plt.scatter(Phase_Angle, Unity)
#plt.ylim(8.1,7)
plt.show()

In [ ]:
plt.scatter(Phase_Angle_Final, Unity_Final)
#plt.ylim(8.1,7)
plt.show()

### 3. Applying Filter Offsets

In [ ]:
### Applying Filter Conversions for ALCDEF data

t2_ALCDEF = []
t1_ALCDEF = Phase_Angle_Final
for i in Unity_Final:
    t2_ALCDEF.append(i+1.18)
nysa = Table([t1_ALCDEF,t2_ALCDEF], names=('phase','V'))
nysa['phase'].unit = 'deg'
nysa['V'].unit = 'mag'
print(nysa)

### 4. Creating H and G Curves

In [ ]:
res = fit_fun(nysa)
print("RMS of H,G-fit is {:2.8f}".format(res.fun))
print("Fitted H={:1.4f} and G={:1.6f}".format(res.x[0],res.x[1]))

In [ ]:
fx=np.linspace(nysa['phase'][0],nysa['phase'][-1],100)
fy=np.array([pf(x,res.x[0],res.x[1]) for x in fx])
#g,p1=plt.subplots(1,1)
plt.figure(figsize=(6, 5))
plt.plot(nysa['phase'],nysa['V'],'bo')
plt.plot(fx,fy,'r')
plt.title('Themis (ALCDEF)')
plt.xlabel('Phase Angle (deg)')
plt.ylabel('R magnitude')
plt.ylim(max(nysa['V']+0.05), min(nysa['V']-0.05))
plt.show()

## COMBINING ALL SKY SURVEYS AND CITIZEN SCIENTIST OBSERVATIONS

In [ ]:
phase_angle_total = []
unity_total = []

for i in range(len(t1_GAIA)):
    phase_angle_total.append(t1_GAIA[i])
    unity_total.append(t2_GAIA[i])
for i in range(len(t1_ZTF)):
    phase_angle_total.append(t1_ZTF[i])
    unity_total.append(t2_ZTF[i])
for i in range(len(t1_ALCDEF[-2:-1])):
    phase_angle_total.append(t1_ALCDEF[i])
    unity_total.append(t2_ALCDEF[i])

In [ ]:
paired_sorted = sorted(zip(phase_angle_total, unity_total))
t1_total, t2_total = zip(*paired_sorted)

In [ ]:
nysa = Table([t1_total[0:-1],t2_total[0:-1]], names=('phase','V'))
nysa['phase'].unit = 'deg'
nysa['V'].unit = 'mag'
print(nysa)

In [ ]:
res = fit_fun(nysa)
print("RMS of H,G-fit is {:2.8f}".format(res.fun))
print("Fitted H={:1.4f} and G={:1.6f}".format(res.x[0],res.x[1]))

In [ ]:
x_specific = nysa['phase'] 
# Calculate fitted y values for these specific x values
fy_specific = np.array([pf(x, res.x[0], res.x[1]) for x in x_specific])
rmse = np.sqrt(np.mean((nysa['V'] - fy_specific) ** 2))

In [ ]:
rmse

In [ ]:
fx=np.linspace(nysa['phase'][0],nysa['phase'][-1],100)
fy=np.array([pf(x,res.x[0],res.x[1]) for x in fx])
#g,p1=plt.subplots(1,1)
plt.figure(figsize=(7, 6))
#plt.plot(nysa['phase'],nysa['V'],'bo')
plt.plot(t1_ZTF,t2_ZTF,'bo', c = '#22b322', marker = '^', markersize=11) #Green
plt.plot(t1_GAIA,t2_GAIA,'bo', c = '#ffd745', marker = '*', markersize=18) #Yellow
plt.plot(t1_ALCDEF[-2:-1],t2_ALCDEF[-2:-1],'bo', c = '#2f94d4', marker = 's', markersize=10) #Blue
plt.legend(['ZTF Sky Survey (Ground Based)', 'GAIA Sky Survey (Space Based)', 'Citizen Scientist Observations (ALCDEF)'], facecolor='w', labelcolor='k', prop = { "size": 12 }, loc ="upper right")
#plt.legend(facecolor='k', labelcolor='w')
plt.plot(fx,fy,'r')
plt.title('Summerfield Asteroid')
plt.xlabel('Phase Angle (Degrees)')
plt.ylabel('Reduced Magnitude (V)')
plt.ylim(max(nysa['V']+0.05), min(nysa['V']-0.1))
plt.show()

In [ ]:
x_specific = nysa['phase'] 
fy_specific = np.array([pf(x, res.x[0], res.x[1]) for x in x_specific])
rmse = np.sqrt(np.mean((nysa['V'] - fy_specific) ** 2))
            

residuals = nysa['V'] - fy_specific
std_dev = np.std(residuals)

within_2_std = np.abs(residuals) <= 1 * std_dev


filtered_ast = nysa[within_2_std]

filtered3_Phase = filtered_ast["phase"]
filtered3_Unity = filtered_ast["V"]
        
f1_ZTF = []
f2_ZTF = []
f1_GAIA = []
f2_GAIA = []

for i in range(len(filtered_ast["phase"])):
    if filtered_ast["phase"][i] in t1_ZTF:
        
        f1_ZTF.append(filtered_ast["phase"][i])
        f2_ZTF.append(filtered_ast["V"][i])

    if filtered_ast["phase"][i] in t1_GAIA:
        
        f1_GAIA.append(filtered_ast["phase"][i])
        f2_GAIA.append(filtered_ast["V"][i])
        

plt.figure(figsize=(6, 4))
#plt.plot(nysa['phase'],nysa['V'],'bo')
f1_ZTF = np.array(f1_ZTF)
f2_ZTF = np.array(f2_ZTF)

#f1_ZTF[30:43] = np.nan
#f2_ZTF[30:43] = np.nan

#f1_ZTF[65:70] = np.nan
#f2_ZTF[65:70] = np.nan

#f1_ZTF[30:80] = np.nan
#f2_ZTF[30:80] = np.nan

#f1_ZTF[100:110] = np.nan
#f2_ZTF[100:110] = np.nan


#t1_ALCDEF = [x - 2 for x in t1_ALCDEF]
#t2_ALCDEF = [x + 0.04 for x in t2_ALCDEF]

plt.plot(f1_ZTF,f2_ZTF,'bo', c = '#22b322', marker = '^', markersize=11) #Green
plt.plot(f1_GAIA,f2_GAIA,'bo', c = '#ffd745', marker = '*', markersize=18) #Yellow
plt.plot(t1_ALCDEF,t2_ALCDEF,'bo', c = '#2f94d4', marker = 's', markersize=10) #Blue

#plt.scatter(nysa[within_2_std]["phase"],nysa[within_2_std]["V"])
#plt.legend(['ZTF Sky Survey', 'GAIA Sky Survey', 'Citizen Scientist Observations (ALCDEF)'], facecolor='w', labelcolor='k', prop = { "size": 15 }, loc ="upper right")
#plt.legend(facecolor='k', labelcolor='w')
plt.plot(fx,fy,'r')
plt.title('(4786) Tatianina', fontsize = 18)
plt.xlabel('Phase Angle (Degrees)', fontsize = 14)
plt.ylabel('Reduced Magnitude', fontsize = 14)
plt.ylim(max(nysa['V'])-0.03, min(nysa['V']-0.07))
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.axvline(x=7, color='gray', linestyle='--', label='7 degrees') 
plt.text(0.5, 0.92, f'H = {round(13.68, 2)} ± {round(rmse,2)}' + f'\n' + f'G = 0.40', 
         horizontalalignment='left', 
         verticalalignment='top', 
         fontsize=17, 
         bbox=dict(facecolor='white', edgecolor='black', boxstyle='Square, pad=0.5'), 
         transform=plt.gca().transAxes)

plt.text(0.046, 0.1, f'Albedo = 0.32',
         horizontalalignment='left', verticalalignment='bottom', 
         fontsize=17, color='darkblue', 
         transform=plt.gca().transAxes)
plt.show()